In [1]:
import numpy as np
import pandas as pd
from sklearn.model_selection import train_test_split
import numpy as np
import pydot
from io import StringIO
from sklearn.tree import export_graphviz
from sklearn.preprocessing import MinMaxScaler
from sklearn.preprocessing import StandardScaler

## For standardization: 
Depending on the task objetives. For example; for neural networks is recommended normalization Min max for activation functions. To avoid saturation Basheer & Najmeer (2000) recommend the range 0.1 and 0.9.

# Prep Model

In [2]:
def getfile():
    df=pd.read_csv(r'D3.csv')
    a=10.986409919681789
    b=171.91508511054587
    df['height']=df['height']*a+b       
    return df

In [13]:
def data_d3(): 
    df = getfile()
    df['height']=df.height.round()
    df['contacts_count']=df.contacts_count.round()
    #Height is Z-score Normalized  => need to change => reverse Z-score and convert to int32
    # contacts_count in assessment 1 have been imputed with MEAN => round it into int32
    mapping = {'yes':1, 'no':0, 'blank':np.nan}
    df['insurance']=df.insurance.map(mapping)
    df['insurance'].fillna(df['insurance'].mode()[0], inplace=True)
    secmap={'native':1,'immigrant':0, 'blank':np.nan}
    df['immigrant']=df.immigrant.map(secmap)
    df['immigrant'].fillna(df['immigrant'].mode()[0], inplace=True)
    #Convert data type: 
    convert_dict = {'contacts_count':int, 'height': int,'worried': int, 'immigrant':bool,'insurance':bool,'covid19_positive':bool,'covid19_symptoms':bool,'covid19_contact':bool,'asthma':bool,'kidney_disease':bool,'liver_disease':bool,'compromised_immune':bool,'heart_disease':bool,'lung_disease':bool,'diabetes':bool,'hiv_positive':bool,'hypertension':bool,'other_chronic':bool,'nursing_home':bool,'health_worker':bool}
    df = df.astype(convert_dict) 
    #Remove outlier
    df = pd.get_dummies(df)
    y = df['covid19_positive']
    X = df.drop(['covid19_positive'], axis=1)
    
    
    # setting random state
    rs = 42

    X_mat = X.to_numpy()
    X_train, X_test, y_train, y_test = train_test_split(X_mat, y, test_size=0.29, stratify=y, random_state=rs)

    return df,X,y,X_train, X_test, y_train, y_test

In [14]:
df=getfile()
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5111 entries, 0 to 5110
Data columns (total 33 columns):
region                    5111 non-null object
country                   5111 non-null object
sex                       5111 non-null object
age                       5111 non-null object
height                    5111 non-null float64
weight                    5111 non-null int64
blood_type                5111 non-null object
insurance                 5111 non-null object
income                    5111 non-null object
race                      5111 non-null object
immigrant                 5111 non-null object
smoking                   5111 non-null object
contacts_count            5111 non-null float64
house_count               5111 non-null int64
public_transport_count    5111 non-null int64
working                   5111 non-null object
worried                   5111 non-null int64
covid19_positive          5111 non-null int64
covid19_symptoms          5111 non-null int64
covi

In [15]:
# For Regression: 
def analyse_feature_importance(dm_model, feature_names, n_to_display=20):
    # grab feature importances from the model
    importances = dm_model.feature_importances_
    
    # sort them out in descending order
    indices = np.argsort(importances)
    indices = np.flip(indices, axis=0)

    # limit to 20 features, you can leave this out to print out everything
    indices = indices[:n_to_display]

    for i in indices:
        print(feature_names[i], ':', importances[i])

In [16]:
# Decision Tree 
def visualize_decision_tree(dm_model, feature_names, save_name):
    dotfile = StringIO()
    export_graphviz(dm_model, out_file=dotfile, feature_names=feature_names)
    graph = pydot.graph_from_dot_data(dotfile.getvalue())
    graph[0].write_png(save_name) # saved in the following file

In [17]:
df,X,y,X_train, X_test, y_train, y_test = data_d3()

In [20]:
display(df.info())

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 5111 entries, 0 to 5110
Columns: 167 entries, height to working_travel non critical
dtypes: bool(17), float64(1), int32(3), int64(3), uint8(143)
memory usage: 1018.3 KB


None

In [19]:
df

,height,weight,insurance,immigrant,contacts_count,house_count,public_transport_count,worried,covid19_positive,covid19_symptoms,...,smoking_quit5,smoking_vape,smoking_yesheavy,smoking_yeslight,smoking_yesmedium,working_home,working_never,working_stopped,working_travel critical,working_travel non critical
0,184,80,True,True,8,5,0,4,True,False,...,0,0,0,0,0,0,0,1,0,0
1,164,84,True,True,8,1,0,4,True,False,...,0,0,0,0,0,0,0,1,0,0
2,170,64,True,True,10,1,0,4,True,False,...,0,0,0,0,0,0,0,1,0,0
3,132,124,True,True,8,10,0,4,True,False,...,1,0,0,0,0,0,0,1,0,0
4,184,118,True,True,8,3,0,4,True,True,...,0,0,0,0,0,0,0,1,0,0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
5106,166,66,True,True,1,2,0,4,False,False,...,0,0,0,0,0,0,1,0,0,0
5107,168,70,True,False,2,3,0,4,False,False,...,0,0,0,0,0,0,1,0,0,0
5108,182,82,True,True,3,3,0,4,False,False,...,0,0,0,0,0,0,1,0,0,0
5109,184,80,True,True,0,3,0,3,False,False,...,0,0,0,0,0,0,0,1,0,0
